In [1]:
from selenium import webdriver
from time import sleep
import json
from selenium.common.exceptions import NoSuchElementException

In [2]:
import requests
from bs4 import BeautifulSoup

driver = webdriver.Chrome('./chromedriver')
# 나중에 이거 주소를 리스트에 넣고 하나씩 반복해서 돌리면 될거같음
# 이거 저기 ms부분만 리스트에 넣고 url주소 바꾸면서 가져오면 될거같음 (ms가 위도경도임)
driver.get('https://new.land.naver.com/complexes?ms=37.5771187,126.9298968,12&a=APT:JGC:ABYG&e=RETAIL')


In [3]:
sleep(3)

content = BeautifulSoup(driver.page_source, 'html.parser')
# 원룸/투룸 클릭 
driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
sleep(1)
# 투룸 제외(원룸만) -> 이거 체크되어있는지 확인하고(area-pressed = true인지 false인지)
# 체크되어있으면 클릭 안하도록 만들기
driver.find_element_by_css_selector("#type1").click()
sleep(1)
driver.find_element_by_css_selector("#list > div > div > div.list_fixed > div > a:nth-child(2)").click()
sleep(1)

In [4]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

actions = ActionChains(driver)

# 페이지 전체를 스크롤다운 하려는게 아니라 옆에 리스트만 스크롤 다운하고 싶기 때문에 이걸 해줌
scrollArea = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
sleep(1)

# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)

# actions.move_to_element(element).perform()
last_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)

try:
    a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
except NoSuchElementException:
    a = None

while(True):
    # 무한스크롤의 끝까지 늘려줌
    # 스크롤이 끝나기 전에는 loader이라는 div가 있는데 그게 더 이상 없을 때까지 스크롤을 내려줌  
    while a != None:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)
        sleep(1)
        try:
            new_a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
        except NoSuchElementException:
            new_a = None
        sleep(1)    
        if new_a != a:
            a = new_a
        
    new_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)
    if new_height == last_height:
        break
    last_height = new_height

print("loading complete")
# new_height = driver.execute_script("arguments[0].scrollTo(0, 0);", scrollArea)
# 스크롤바 제일 위로 올라감
driver.execute_script("arguments[0].scrollTo(0,0);", scrollArea)
sleep(3)

loading complete


In [15]:
content = BeautifulSoup(driver.page_source, 'html.parser')

# span class명 정리
# area is-selected: xx시 xx구 xx동
# text: 건물명
# type: 월세/전세를 구분함
# spec: 공급면적/전용면적, 해당층/총층, 집 방향(ex.남동향 등등)
# price: 월세/전세의 가격. 월세면 보증금/월세 의 형태로 나타남
# strong clas명 정리
# type: 건물 형태(ex.아파트,원룸 등등)

# em class명 정리
# title: 확인 = 아직 거래가 되지 않은 매물
#        거래완료 = 거래가 완료된 매물

# 나중에 아이템을 하나씩 찾도록 바꾸기

# list_items = content.findAll('span',{'class':["area is-selected","type","price"]})
# list_items_a = content.select('#region_filter > div > a > span')
# list_items_b = []
# list_items_c = []
# for item in list_items_a:
#     list_items_b.append(item.text)
    
# for item in list_items:
#     list_items_c.append(item.text)

city = []
Gu = []
Dong = []
place = []
list_area = content.select('span.area.is-selected')
for item in list_area:
    place.append(item.text)
city.append(place[0])
Gu.append(place[1])
Dong.append(place[2])
print(place)
    

# 건물명
names = []
list_name = content.select('span.text')
for item in list_name:
    names.append(item.text)
    
# 월세/전세
types = []
list_type = content.select('span.type')
for item in list_type:
    types.append(item.text)

# 가격
prices = []
list_price = content.select('span.price')
for item in list_price:
    prices.append(item.text)
    
# 건물 형태
buildings = []
list_build = content.select('strong.type')
for item in list_build:
    buildings.append(item.text)

# 기타 정보
specifics = []
list_spec = content.select('span.spec')
for item in list_spec:
    specifics.append(item.text)

# 판매여부
checks = []
list_check = content.select('em.title')
for item in list_check:
    checks.append(item.text)

print(city)
print(Gu)
print(Dong)
print(names)
print(types)
print(prices)
print(buildings)
print(specifics)
print(checks)
# class House:
#     def __init__(self, payment, price):
#         self.payment = payment
#         self.price = price
    
#     def __str__(self):
#         return self.payment + " = " + self.price
    
#     def __repr__(self):
#         return str(self)
# print(list_items)    
# print(list_items_b)
# print(list_items_c)

['서울시', '서대문구', '연희동']
['서울시']
['서대문구']
['연희동']
['일반원룸 서일빌 ', '일반원룸 ', '일반원룸 ', '일반원룸 JOY HOUSE ', '일반원룸 ', '정연지오빌 1동', '정연지오빌 1동', '시아루비움 1동', '일반원룸 ', '다가구 백년주택 B동 ', '단독 ', '빌라 ', '정연지오빌 1동', '연희웨스트팰리스(도시형) 1동', '연희미다스파크 104동', '일반원룸 JOY HOUSE ', '일반원룸 JOY HOUSE ', '일반원룸 ', '일반원룸 ', '파라다이스텔SB 1동', '파라다이스텔SB 1동', '일반원룸 ', '기타 ', '일반원룸 원광빌라 ', '일반원룸 ', '연립 ', '일반원룸 ', '더연희힐(도시형) 1동', '일반원룸 ', '일반원룸 ', '다가구 ', '일반원룸 그린 하우스 ', '연희브라운스톤(도시형) B동', '연희소프트빌 1동', '일반원룸 ', '일반원룸 ', '오벨리스크 1동', '일반원룸 ', '일반원룸 Time Square ', '일반원룸 영원해피하우스 ', '일반원룸 ', '기타 우리집 ', '단독 ', '스페이스엠 1동', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '오벨리스크 1동', '파라다이스텔SB 1동', '일반원룸 ', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '다가구 ', '정연지오빌 1동', '정연지오빌II 1동', '일반원룸 ', '오피스텔 ', '빌라 ', '연희웨스트팰리스(도시형) 1동', '다가구 ', '다가구 ', '일반원룸 ', '정연지오빌II 1동', '빌라 디타운하우스 ', '일반원룸 Y-HOUSE ', '오벨리스크 1동', '오피스텔 ', '다가구 백년주택 B동 ', '다가구 ', '일반원룸 ', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '연희자이엘라(도시형) 1동', '시아루비움 1동', '시아루비움 1동', '일반원룸 힐탑하우스  ', '일반원룸 ', '하순', '빌

In [21]:
items = []
for item in list_items:
    item_payment = item.type
    item_price = item.price
    
    if item_payment is not None and item_price is not None:
        new_item = House(item_payment.text, item_price.text)
        items.append(new_item)
        
print(items)
    

[]


In [7]:
house = []
for item in range(len(list_items_a)): 
    new = House(list_items_a[item], list_items_b[item])
    house.append(new)

print(house)

TypeError: unsupported operand type(s) for +: 'Tag' and 'str'